# Apache Spark

## Apache Spark™ is a unified analytics engine for large-scale data processing.
![](https://spark.apache.org/images/spark-logo-trademark.png)

# Features

## Speed

![](https://cc-media-foxit.fichub.com/image/fox-it-mondofox/0177f439-3c0f-44ae-9803-c25f8bfac0dd/flash-vs-superman-game-2jpg-maxw-824.jpg)

### Run workloads 100x faster.

![Logistic Regression](https://spark.apache.org/images/logistic-regression.png)

Apache Spark achieves 

- high performance for both batch and streaming data

- using a state-of-the-art DAG scheduler

- a query optimizer

- a physical execution engine.

## Why Spark is faster ?

## In - Memory

![](https://external-preview.redd.it/RVpCIxhliY2p5vKF8I-AoCLIoI48yIEpVPXDduTG6Fc.jpg?auto=webp&s=88a001359893e5533423e9886d4d55cfd2dbdf62)

## Lazy

![](https://miro.medium.com/max/4096/1*KiC1gf3x3Ia_2PBYqfkLBg.jpeg)

## Ease of Use

![](http://www.quickmeme.com/img/4d/4d4759d82ce65de86834ff151bc8b419f89f4e2f0d003f10a54b236785e3e6d2.jpg)

### Write applications quickly in Java, Scala, Python, R, and SQL.

Spark offers over 80 high-level operators that make it easy to build parallel apps. 

And you can use it **interactively** from the Scala, Python, R, and SQL shells.

#### Scala Example


```scala
df = spark.read.json("logs.json") 
df.where("age > 21").select("name.first").show()
```

## Generality

![](https://upload.wikimedia.org/wikipedia/commons/3/3d/%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%B0_%D0%B3%D0%BE%D0%B4%D0%B0._%D0%94%D0%B6%D1%83%D0%B7%D0%B5%D0%BF%D0%BF%D0%B5_%D0%90%D1%80%D1%87%D0%B8%D0%BC%D0%B1%D0%BE%D0%BB%D1%8C%D0%B4%D0%BE.jpg)

> Arcimboldo - 1563 - Quattro stagioni

### Combine SQL, streaming, and complex analytics.

Spark powers a stack of libraries including 

- SQL and DataFrames

- MLlib for machine learning

- GraphX

- Spark Streaming. 

You can combine these libraries seamlessly in the same application.

![](https://spark.apache.org/images/spark-stack.png)

## Runs everywhere

![](https://www.nextme.it/images/societa/next-economy/Criceti_umani.jpg)

### Spark runs on Hadoop, Apache Mesos, Kubernetes, standalone, or in the cloud. 

You can run Spark using its standalone cluster mode, on EC2, on Hadoop YARN, on Mesos, or on Kubernetes
![](https://spark.apache.org/images/spark-runs-everywhere.png)

### It can access diverse external data sources


#### Analyse
Spark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.

https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets

#### Query
Spark SQL supports operating on a variety of data sources through the DataFrame interface. A DataFrame can be operated on using relational transformations and can also be used to create a temporary view. Registering a DataFrame as a temporary view allows you to run SQL queries over its data.

https://spark.apache.org/docs/latest/sql-data-sources.html

# Overview


Apache Spark is a fast and general-purpose cluster computing system. It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs. It also supports a rich set of higher-level tools including Spark SQL for SQL and structured data processing, MLlib for machine learning, GraphX for graph processing, and Spark Streaming.

## Docker

```Dockerfile
FROM openjdk:8-jre
ENV PATH $SPARK_DIR/bin:$PATH
ENV SPARK_VERSION=3.2.1
ENV SPARK_DIR=/opt/spark
ENV PATH $SPARK_DIR/bin:$PATH

ADD setup/spark-${SPARK_VERSION}-bin-hadoop3.2.tgz /opt

RUN apt-get update && apt-get -y install bash python3 python3-pip netcat

RUN pip3 install pyspark numpy elasticsearch
# Create Sym Link 
RUN ln -s /opt/spark-${SPARK_VERSION}-bin-hadoop3.2 ${SPARK_DIR} 

ADD dataset /opt/tap/spark/dataset
# Add Python Code
ADD code/*  /opt/tap/
# Add Java Code
ADD apps /opt/tap/apps
# Add Spark Manager
ADD spark-manager.sh $SPARK_DIR/bin/spark-manager

WORKDIR ${SPARK_DIR}
ENTRYPOINT [ "spark-manager" ]
```

# Spark Manager

```bash
#!/bin/bash
[[ -z "${SPARK_ACTION}" ]] && { echo "SPARK_ACTION required"; exit 1; }

# ACTIONS start-zk, start-kafka, create-topic, 

echo "Running action ${SPARK_ACTION}"
case ${SPARK_ACTION} in
"example")
echo "Running example ARGS $@"
./bin/run-example $@
;;
"spark-shell")
./bin/spark-shell --master local[2]
;;
"pyspark")
./bin/pyspark --master local[2]
;;
"spark-submit-python")
 ./bin/spark-submit --packages $2 /opt/tap/$1
;;
"spark-submit-apps")
 ./bin/spark-submit --packages $3 --class $1 /opt/tap/apps/$2
;;
"pytap")
cd /opt/tap/
python ${TAP_CODE}
;;
"bash")
while true
do
	echo "Keep Alive"
	sleep 10
done
;;
esac

```


## SparkPI
https://github.com/apache/spark/blob/master/examples/src/main/python/pi.py

Use Monte Carlo Method  https://theabbie.github.io/blog/estimate-pi-using-random-numbers.html

Run sparkExamplePi.sh
```bash
#!/usr/bin/env bash
# Stop
docker stop sparkPi

# Remove previuos container 
docker container rm sparkPi

docker build ../spark/ --tag tap:spark
docker run -e SPARK_ACTION=example --network tap --name sparkPi -it tap:spark SparkPi 10
```

```
Running action example
Running example ARGS SparkPi 100
21/04/18 15:31:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/04/18 15:31:59 INFO SparkContext: Running Spark version 3.1.1
21/04/18 15:31:59 INFO ResourceUtils: ==============================================================
21/04/18 15:31:59 INFO ResourceUtils: No custom resources configured for spark.driver.
21/04/18 15:31:59 INFO ResourceUtils: ==============================================================
21/04/18 15:31:59 INFO SparkContext: Submitted application: Spark Pi

...
Pi is roughly 3.1419099141909914
21/04/18 15:32:09 INFO SparkUI: Stopped Spark web UI at http://958a11429922:4040
21/04/18 15:32:09 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
21/04/18 15:32:09 INFO MemoryStore: MemoryStore cleared
21/04/18 15:32:09 INFO BlockManager: BlockManager stopped
21/04/18 15:32:09 INFO BlockManagerMaster: BlockManagerMaster stopped
21/04/18 15:32:09 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
21/04/18 15:32:09 INFO SparkContext: Successfully stopped SparkContext
21/04/18 15:32:09 INFO ShutdownHookManager: Shutdown hook called
21/04/18 15:32:09 INFO ShutdownHookManager: Deleting directory /tmp/spark-bc591bec-ca63-4e7b-86f4-191684261e8f
21/04/18 15:32:09 INFO ShutdownHookManager: Deleting directory /tmp/spark-4dc0f0c2-7124-4fc7-9122-b4ea4789f57f
```


# Spark Shell

```bash
./sparkShell.sh
### docker run -e SPARK_ACTION=spark-shell --network tap -it tap:spark)
```

```
Spark context Web UI available at http://17827060ca34:4040
Spark context available as 'sc' (master = local[2], app id = local-1586889603227).
Spark session available as 'spark'.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Scala version 2.11.12 (OpenJDK 64-Bit Server VM, Java 1.8.0_212)
Type in expressions to have them evaluated.
Type :help for more information.

scala>
```


```scala
scala> val textFile=spark.read.textFile("/opt/tap/spark/dataset/lotr_characters.csv");
scala> textFile.count();
res0: Long = 912

scala> textFile.first();

res3: String = birth,death,gender,hair,height,name,race,realm,spouse

```


# Spark Data Structures

# Resilient Distributed Dataset (RDD)

> https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds

Spark revolves around the concept of a resilient distributed dataset (RDD)

![](https://1.bp.blogspot.com/-wMroEy8Ow-k/WdCUxRefTTI/AAAAAAAABNM/Z14px-DgqGYqPfAfwNIILI9EX-ozLGplQCLcBGAs/s640/apache-spark-streaming-13-638.jpg)

fault-tolerant 

![](https://i.imgflip.com/1dzjjc.jpg)

collection of elements

![](https://mallikarjuna_g.gitbooks.io/spark/content/diagrams/spark-rdds.png)

> https://books.japila.pl/apache-spark-internals/

that can be operated on in parallel. 

![](https://mallikarjuna_g.gitbooks.io/spark/content/diagrams/spark-rdd-partitioned-distributed.png)

> Learning Spark 

An RDD in Spark is simply an immutable distributed collection of objects. 

Each RDD is split into multiple partitions, which may be computed on different nodes of the cluster. 

RDDs can contain any type of Python, Java, or Scala objects, including user-defined classes.

There are two ways to create RDDs

parallelizing an existing collection in your driver program, 

referencing a dataset in an external storage system, 
such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat

# Py Spark

# Start Py Spark Docker
```bash
./pyspark.sh 
```

In [ ]:
export JAVA_HOME=/Library/Java/JavaVirtualMachines/zulu-8.jdk/Contents/Home

In [1]:
import findspark
import pyspark
findspark.find() 
findspark

<module 'findspark' from '/Users/nics/miniforge3/lib/python3.9/site-packages/findspark.py'>

In [2]:
conf = pyspark.SparkConf().setAppName('Tap').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/10 21:25:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=Tap>

In [3]:
# List
data = [1, 2, 3, 4, 5] 
data

[1, 2, 3, 4, 5]

In [4]:
distData = sc.parallelize(data)
distData

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [5]:
# An RDD can be also created from external storage
# textFile creates a RDD(String) (remember when we use spark.read.file)
distFile = sc.textFile("/Users/nics/Dev/GitHub/tap2022/spark/dataset/The Return Of The King_djvu.txt") # Path may be different in your local env
distFile

/Users/nics/Dev/GitHub/tap2022/spark/dataset/The Return Of The King_djvu.txt MapPartitionsRDD[2] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
sizeOfBook=distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)
sizeOfBook

710716

# Key Pairs

While most Spark operations work on RDDs containing any type of objects, a few special operations are only available on RDDs of key-value pairs. The most common ones are distributed “shuffle” operations, such as grouping or aggregating the elements by a key.

In Python, these operations work on RDDs containing built-in Python tuples such as (1, 2). Simply create such tuples and then call your desired operation.

In [7]:
pairs = distFile.map(lambda s: (s, 1))
pairs

PythonRDD[4] at RDD at PythonRDD.scala:53

We have create a new RDD, let's see what it contains

In [8]:
pairs.take(20)

[('"trti ', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('“THE LORD OF THE RINGS” ', 1),
 ('', 1),
 ('Pjrt Thttt ', 1),
 ('', 1),
 ('THE RETURN ', 1),
 ('OF THE KING ', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('J.R.R.ToIkien ', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('* BOOK V * ', 1),
 ('', 1)]

Now we can use a reduce function, to count how may times the line appears in the document

In [9]:
counts = pairs.reduceByKey(lambda a, b: a + b)
counts

PythonRDD[10] at RDD at PythonRDD.scala:53

In [14]:
counts.take(10)

[('', 5807),
 ("“THE LORD OF THE RINGS' ", 1),
 ('V*art One ', 1),
 ('THE FELLOWSHIP ', 1),
 ('OF THE RING ', 1),
 ('J.R.R.ToIkien ', 1),
 ('Complete Table of Contents ', 1),
 ('Foreword ', 2),
 ('Prologue ', 1),
 ('1 . Concerning Hobbits ', 1)]

Let's order by key

In [15]:
ordered=counts.sortByKey()

In [16]:
ordered.takeOrdered(10)

[('', 5807),
 ('" "The enemy must have some great need or purpose," said Radagast; "but ',
  1),
 ('" "The time of my thought is my own to spend," answered Dbin. ', 1),
 ('"\'Give us that, Deal, my love," said Smjagol, over his friend\'s ', 1),
 ('"Ass! Fool! Thrice worthy and beloved Barliman! " said I. "It\'s the ', 1),
 ('"At the worst," said he, "our Enemy knows that we have it not and ', 1),
 ('"I am afraid we must go back to the Road here for a while,\' said ', 1),
 ('"I will do that," he said, and rode off as if the Nine were after ', 1),
 ('"So you have come, Gandalf," he said to me gravely; but in his eyes ', 1),
 ('"Talking,\' said Bilbo. \'There was a deal of talk, and everyone had an ',
  1)]

# Let's do a better analysis
Which is the most frequent word in the book ?

In [29]:
words=distFile.flatMap(lambda line:line.split(" "))

In [30]:
words.take(100)

['"trti',
 '',
 '',
 '',
 '',
 '“THE',
 'LORD',
 'OF',
 'THE',
 'RINGS”',
 '',
 '',
 'Pjrt',
 'Thttt',
 '',
 '',
 'THE',
 'RETURN',
 '',
 'OF',
 'THE',
 'KING',
 '',
 '',
 '',
 '',
 'J.R.R.ToIkien',
 '',
 '',
 '',
 '',
 '',
 '*',
 'BOOK',
 'V',
 '*',
 '',
 '',
 '',
 '',
 'Chapter',
 '1',
 '.',
 'Minas',
 'Tirith',
 '',
 '',
 '',
 '',
 'Pippin',
 'looked',
 'out',
 'from',
 'the',
 'shelter',
 'of',
 'Gandalf',
 's',
 'cloak.',
 'He',
 'wondered',
 'if',
 '',
 'he',
 'was',
 'awake',
 'or',
 'still',
 'sleeping,',
 'still',
 'in',
 'the',
 'swift',
 '-moving',
 'dream',
 'in',
 'which',
 'he',
 '',
 'had',
 'been',
 'wrapped',
 'so',
 'long',
 'since',
 'the',
 'great',
 'ride',
 'began.',
 'The',
 'dark',
 'world',
 'was',
 '',
 'rushing',
 'by',
 'and',
 'the',
 'wind',
 'sang']

Great, let's assign a counter and then sum 

In [31]:
wordCounters=words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

In [20]:
wordCounters.take(10)

[('', 21555),
 ('“THE', 1),
 ('LORD', 1),
 ('OF', 3),
 ('THE', 9),
 ("RINGS'", 1),
 ('V*art', 1),
 ('One', 67),
 ('FELLOWSHIP', 1),
 ('RING', 1)]

Ok I want to sort now

In [32]:
wordsSorted=wordCounters.takeOrdered(200, key = lambda x: -x[1])
wordsSorted

[('', 14916),
 ('the', 8035),
 ('and', 5811),
 ('of', 3964),
 ('to', 2768),
 ('a', 2337),
 ('in', 2004),
 ('he', 1923),
 ('that', 1644),
 ('was', 1505),
 ('his', 1301),
 ('I', 1269),
 ('it', 1081),
 ('they', 1041),
 ('you', 990),
 ('for', 903),
 ('as', 898),
 ('not', 890),
 ('with', 868),
 ('said', 847),
 ('had', 810),
 ('is', 805),
 ('at', 759),
 ('all', 696),
 ('on', 688),
 ('have', 648),
 ('be', 646),
 ('but', 635),
 ('were', 617),
 ('from', 594),
 ('And', 552),
 ('But', 531),
 ('will', 516),
 ('their', 484),
 ('there', 482),
 ('The', 469),
 ('now', 451),
 ('no', 423),
 ('came', 422),
 ('if', 408),
 ('or', 404),
 ('great', 401),
 ('we', 396),
 ('He', 394),
 ('my', 367),
 ('are', 362),
 ('by', 357),
 ('him', 353),
 ('out', 351),
 ('up', 347),
 ('would', 319),
 ('your', 317),
 ('them', 307),
 ('could', 305),
 ('this', 298),
 ('into', 291),
 ('like', 287),
 ('upon', 275),
 ('then', 266),
 ('when', 260),
 ('one', 258),
 ('so', 257),
 ('been', 256),
 ('long', 256),
 ('more', 254),
 ('som

In [33]:
sc.stop()

# Biblio
- https://www.kdnuggets.com/2017/08/three-apache-spark-apis-rdds-dataframes-datasets.html
- https://www.slideshare.net/differentsachin/apache-spark-introduction-and-resilient-distributed-dataset-basics-and-deep-dive
- https://data-flair.training/blogs/apache-spark-rdd-vs-dataframe-vs-dataset/
- https://www.slideshare.net/taposhdr/resilient-distributed-datasets
- http://vishnuviswanath.com/spark_rdd.html
- https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-actions/
- https://www.educba.com/rdd-in-spark/
- https://www.javahelps.com/2019/02/spark-03-understanding-resilient.html
- https://www.tutorialspoint.com/apache_spark/apache_spark_rdd.htm
- https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf
- https://www.sicara.ai/blog/2017-05-02-get-started-pyspark-jupyter-notebook-3-minutes